In [1]:
## fix dataset first 

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('..\\..\\preprocessed\\tweets_general.csv')
df.dropna(inplace=True)
df.head(2)

,tweet,tokens,label
0,woman shouldnt complain cleaning house man alw...,"['woman', 'shouldnt', 'complain', 'cleaning', ...",0
1,boy dat coldtyga dwn bad cuffin dat hoe st place,"['boy', 'dat', 'coldtyga', 'dwn', 'bad', 'cuff...",0


### data preparation

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

tweets = df['tweet'].tolist()
labels = df['label'].tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(tweets) ## tokenizer fit to our vocabulary

tweet_sequences = tokenizer.texts_to_sequences(tweets)

max_sequence_length = 55  ## padding to ensure uniformity in length of the sequences
tweet_padded = pad_sequences(tweet_sequences, maxlen=max_sequence_length, padding='post')
print(tweet_padded[:2])
labels = np.array(labels)

[[  98  845  936 2989  202   35   83   68   15    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]
 [  95   75 8400 5937   34 2077   75    2  447  379    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]]


In [6]:
## the cnn model 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

embedding_dim = 55 
vocab_size = len(tokenizer.word_index) + 1

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

c:\Users\bemne\anaconda3\envs\al\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(tweet_padded, labels, test_size=0.2, random_state=42)

# Train the model
epochs = 3 # Adjust as needed
batch_size = 32  # Adjust based on your hardware capabilities

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))


Epoch 1/3
620/620 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9376 - loss: 0.2297 - val_accuracy: 0.9431 - val_loss: 0.1494
Epoch 2/3
620/620 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9539 - loss: 0.1239 - val_accuracy: 0.9405 - val_loss: 0.1684
Epoch 3/3
620/620 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9776 - loss: 0.0608 - val_accuracy: 0.9348 - val_loss: 0.2277


In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 55, 55)         │     1,076,515 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 51, 128)        │        35,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,360,494 (12.82 MB)

 Trainable params: 1,120,164 (4.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,240,330 (8.55 MB)

Trying the test on a balanced dataset

In [9]:
from sklearn.utils import resample

df_majority = df[df.label == 0]
df_minority = df[df.label == 1]

# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                   replace=False,    # sample without replacement
                                   n_samples=len(df_minority),  # to match minority class
                                   random_state=42)  # reproducible results

df_balanced = pd.concat([df_minority, df_majority_downsampled])
df_balanced.dropna(inplace=True)
print(df_balanced['label'].value_counts())

tweets_balanced = df_balanced['tweet'].tolist()
labels_balanced = df_balanced['label'].tolist()

tweet_sequences_balanced = tokenizer.texts_to_sequences(tweets_balanced)

tweet_padded_balanced = pad_sequences(tweet_sequences_balanced, maxlen=max_sequence_length, padding='post')
print(tweet_padded_balanced[:2])
labels_balanced = np.array(labels_balanced)


label
1    1430
0    1430
Name: count, dtype: int64
[[ 302 8429    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]
 [8431  130  732  144   64  130  783    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0]]


In [10]:
## new model for balanced df

model_balanced = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_sequence_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_balanced.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model_balanced.summary()

c:\Users\bemne\anaconda3\envs\al\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
## train cnn on balanced df and evaluate

X_train_balanced, X_val_balanced, y_train_balanced, y_val_balanced = train_test_split(tweet_padded_balanced, labels_balanced, test_size=0.2, random_state=42)

model_balanced.fit(X_train_balanced, y_train_balanced, epochs=7, batch_size=batch_size, validation_data=(X_val_balanced, y_val_balanced))


Epoch 1/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5718 - loss: 0.6782 - val_accuracy: 0.7378 - val_loss: 0.5423
Epoch 2/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8290 - loss: 0.4214 - val_accuracy: 0.7972 - val_loss: 0.4696
Epoch 3/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9412 - loss: 0.1891 - val_accuracy: 0.7955 - val_loss: 0.5569
Epoch 4/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.9811 - loss: 0.0710 - val_accuracy: 0.7902 - val_loss: 0.6560
Epoch 5/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9890 - loss: 0.0372 - val_accuracy: 0.7762 - val_loss: 0.7421
Epoch 6/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9944 - loss: 0.0211 - val_accuracy: 0.7675 - val_loss: 0.7996
Epoch 7/7
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9965 - loss: 0.0125 - val_accuracy: 0.7552 - val_loss: 0.9161


In [16]:
from sklearn.metrics import classification_report
import numpy as np

# Predict classes using the model
y_val_pred_probs_balanced = model_balanced.predict(X_val_balanced)
threshold = 0.5

# Convert prediction probabilities to class labels based on the threshold
y_val_pred_balanced = [1 if prob > threshold else 0 for prob in y_val_pred_probs_balanced]

# Print the classification report
print(classification_report(y_val_balanced, y_val_pred_balanced))


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
              precision    recall  f1-score   support

           0       0.71      0.83      0.77       277
           1       0.81      0.69      0.74       295

    accuracy                           0.76       572
   macro avg       0.76      0.76      0.75       572
weighted avg       0.76      0.76      0.75       572



We can see from their outputs that:

-- The model performs well initially on validation on the **entire dataset** but starts to overfit as the  validation loss is increasing despite high training accuracy.

__ might need regularization, in progress.

-- The model's performance on the **balanced dataset** increases for one epoch thens starts decreasing. it's overfitting and needs regularization/ fine tuning.